# Homework 3

## Introduction
This objective of this program focuses on constructing an evolutionary tree using the UPGMA (Unweighted Pair Group Method with Arithmetic Mean) algorithm to examine the fundamentals of graph traversal. UPGMA is a key bioinformatics tool for analyzing genetic relationships by clustering sequences based on their edit distances. Starting with a set of sequences from Neanderthals, humans, and primates, we leverage dynamic programming to compute pairwise edit distances and organize them into a distance matrix. The UPGMA algorithm then iteratively merges the closest clusters, updating the matrix until all sequences are grouped into a single tree. The final output, a list of clusters, represents the inferred evolutionary relationships. Through the implementation of these algorithms, we discern the role of computational methods in understanding genetic evolution and demonstrate UPGMA's effectiveness in phylogenetic analysis.

In [2]:
import pandas as pd
import numpy as np

## Algorithm Implementation

### Edit Distance
Here we first implement the edit distance algorithm that leverages dynamic programming in order to reduce computational strain. Similar to previous implementation, we save essential edit distance values in a matrix of size (n + 1) x (m + 1) with n being the length of the input string and m being the length of the desired string. We then fill the top row and left-most column of the matrix according to the deletion costs and insertion costs respectively. We use those values to calculate the remaining values within the matrix and find the total edit distance as the matrix value [n+1, m+1], which we return.

In [3]:
# Create the edit distance algorithm
def edit_distance(x, y):
    n = len(x)
    m = len(y)
    
    # Create a matrix of size (n + 1) x (m + 1)
    opt = [[0] * (m + 1) for _ in range (n + 1)]
    
    # Base cases: if y is empty or x is empty
    for i in range(n + 1):
        opt[i][0] = i
    for j in range(m + 1):
        opt[0][j] = j
        
    # Compute edit distances
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if x[i - 1] == y[j - 1]:
                cost = 0 # Cost if they are the same
            else:
                cost = 1 # Cost for substitution
                
            opt[i][j] = min(opt[i - 1][j - 1] + cost, # Cost for substitution
                            opt[i - 1][j] + 1, # Cost for deletion
                            opt[i][j - 1] + 1) # Cost for addition
    
    return opt[n][m] # Edit distance is at the corner of the matrix

### UPGMA Algorithm
The UPGMA algorithm is the main focus of this program and is what we utilize in order to construct our evolutionary tree. We initialize the UPGMA algorithm as a class, with each genetic sequence as a separate cluster as well as the edit distance matrix. Following this, we iteratively find the species pair with the smallest edit distance, merge the two, and update the rest of the distance matrix using the average distance formula. We also track these merges to construct a tree data structure in which each merge creates an edges from the new cluster to the merged clusters. Throughout this process, we record each step and updated distance matrix for later reference. Aside from the UPGMA class implementation that includes methods to find the minimum edit distance pair and merge clusters, we create functions to output the tree edges, algorithm history, and final tree, with a utility function to generate matrix labels.

In [4]:
# Create the UPGMA class of clusters that are represented with a distance matrix
class UPGMA:
    def __init__(self, labels, matrix):
        self.labels = labels.copy()
        self.matrix = [row[:] for row in matrix]
        self.edges = []
        self.history = []
        self.heights = {label: 0 for label in labels}
        self.current_clusters = labels.copy()
        
        # Initialize distance matrix
        self.dist_matrix = {}
        n = len(labels)
        for i in range(n):
            a = labels[i]
            self.dist_matrix[a] = {}
            for j in range(n):
                b = labels[j]
                self.dist_matrix[a][b] = matrix[i][j]
    
    # Create method to find the minimum edit distance in the matrix
    def find_min_pair(self):
        # Set the minimum distance to infinity
        min_dist = float('inf')
        a = b = None
        clusters = self.current_clusters
        
        # Iterate over the matrix and clusters
        for i in range(len(clusters)):
            for j in range(i+1, len(clusters)):
                c1 = clusters[i]
                c2 = clusters[j]
                if self.dist_matrix[c1][c2] < min_dist:
                    min_dist = self.dist_matrix[c1][c2]
                    a, b = c1, c2
        
        # Return the position and value of the minimum distance
        return a, b, min_dist
    
    # Create method to iteratively merge the clusters on the minimum edit distance
    def merge_clusters(self, a, b, distance):
        new_name = ''.join(sorted([a, b]))
        new_height = distance / 2
        self.heights[new_name] = new_height
        
        # Add edges
        self.edges.append((new_name, a, new_height - self.heights[a]))
        self.edges.append((new_name, b, new_height - self.heights[b]))
        
        # Update current_clusters
        self.current_clusters = [c for c in self.current_clusters if c not in (a, b)]
        self.current_clusters.append(new_name)
        
        # Compute new distances
        new_dist = {}
        for c in self.current_clusters[:-1]:  # all except new_name
            d_ac = self.dist_matrix[a][c]
            d_bc = self.dist_matrix[b][c]
            new_dist[c] = (d_ac + d_bc) / 2
        
        # Update dist_matrix
        self.dist_matrix[new_name] = new_dist.copy()
        for c in new_dist:
            self.dist_matrix[c][new_name] = new_dist[c]
        
        # Remove a and b from dist_matrix
        del self.dist_matrix[a]
        del self.dist_matrix[b]
        
        # Remove a and b from other clusters' entries
        for c in self.dist_matrix:
            if a in self.dist_matrix[c]:
                del self.dist_matrix[c][a]
            if b in self.dist_matrix[c]:
                del self.dist_matrix[c][b]
        
        # Record steps in history
        step = {
            'merged': (a, b),
            'new_cluster': new_name,
            'new_dist_matrix': {row: {col: self.dist_matrix[row][col] for col in self.dist_matrix[row]} for row in self.dist_matrix}
        }
        self.history.append(step)
    
    # Create final method to run the UPGMA algorithm
    def run(self):
        while len(self.current_clusters) > 1:
            a, b, min_dist = self.find_min_pair()
            self.merge_clusters(a, b, min_dist)
        return self.edges, self.history

# Create function that prints all the edges of the tree
def print_edges(edges):
    print("Edges of the tree:")
    for parent, child, length in edges:
        print(f"{parent} -- {child}: {length:.1f}")

# Create function that prints the documentation history
def print_documentation(history):
    for i, step in enumerate(history, 1):
        print(f"\nStep {i}:")
        a, b = step['merged']
        new_cluster = step['new_cluster']
        print(f"Merged clusters {a} and {b} into {new_cluster}.")
        print("Updated distance matrix:")
        clusters = sorted(step['new_dist_matrix'].keys())
        
        # Prepare header
        header = ["Cluster"] + clusters
        print("\t".join(header))
        
        # Prepare each row
        for row in clusters:
            row_data = [row]
            for col in clusters:
                if row == col:
                    row_data.append(0)
                else:
                    distance = step['new_dist_matrix'][row].get(col, '')
                    row_data.append(distance)
            
            # Convert to strings
            formatted_row = []
            for item in row_data:
                if isinstance(item, float):
                    formatted_row.append(f"{item:.1f}")
                else:
                    formatted_row.append(str(item))
            print("\t".join(formatted_row))

# Create function to output the final list of clusters representing the tree
def build_final_tree(edges):
    # Build parent-to-children mapping
    parent_to_children = {}
    for parent, child, _ in edges:
        if parent not in parent_to_children:
            parent_to_children[parent] = []
        parent_to_children[parent].append(child)
    
    # Find root (the node not listed as a child)
    all_parents = set(parent_to_children.keys())
    all_children = set()
    for children in parent_to_children.values():
        all_children.update(children)
    root = (all_parents - all_children).pop() if (all_parents - all_children) else None
    
    # Recursively build tree structure
    def _build_tree(node):
        if node not in parent_to_children:
            return node
        children = parent_to_children[node]
        return [_build_tree(child) for child in children]
    
    return _build_tree(root) if root else None

# Create function to generate matrix labels 
def alpha_labels(start, end):
    labels = []
    for i in range(ord(start), ord(end)+1):
        labels.append(chr(i))
    return labels

## Application
First, we apply our edit distance algorithm on real life data, specifically genetic sequence data from the National Center for Biotechnology Information. In our study, we compare the edit distances for each distinct pair within a group of 12 species (12C2 or 66 pairs). We store these edit distances in a symmetric matrix to lay the groundwork for our UPGMA algorithm.


In [9]:
# Created my own CSV file because BioPython library did not work
df = pd.read_csv('Species Data.csv')
df['DNA Sequence'] = df['DNA Sequence'].astype(str).str.replace('\n', '', regex=False)
sequences = df['DNA Sequence'].values

# Calculate the edit distance for each distinct pair within the species data and create matrix
n = len(sequences)
editDistances = [[0] * (n) for _ in range (n)]
for i in range(n):
    for j in range(n):
        editDistances[i][j] = (edit_distance(sequences[i], sequences[j]))

In [10]:
# Print initial edit distance matrix
print(np.matrix(editDistances))

[[  0  46  48 148 106 134 135 146 151 103 156  97]
 [ 46   0  73 163  80 115 110 167 165  77 126  73]
 [ 48  73   0 141  88 135 132 138 140  88 166  92]
 [148 163 141   0 159 178 176  59  18 157 142 154]
 [106  80  88 159   0 122 115 172 160  21 115  42]
 [134 115 135 178 122   0  11 176 181 118 166 119]
 [135 110 132 176 115  11   0 178 180 111 167 112]
 [146 167 138  59 172 176 178   0  60 167 143 167]
 [151 165 140  18 160 181 180  60   0 157 143 157]
 [103  77  88 157  21 118 111 167 157   0 107  36]
 [156 126 166 142 115 166 167 143 143 107   0 108]
 [ 97  73  92 154  42 119 112 167 157  36 108   0]]


After having created the edit distance matrix, we run the UPGMA algorithm on the labeled matrix. We print the edges representing the merges, with lengths calculated based on the height difference between the new cluster and the merged clusters. We also print each merge step, including the updated distance matrix in a formatted table to show the clustering process.

In [22]:
# Run UPGMA algorithm on the edit distance matrix and tree
upgma = UPGMA(M_labels, editDistances)
edges, history = upgma.run()
    
# Print tree edges and step by step merging process
print_edges(edges)
print("\nDocumentation of the Process:")
print_documentation(history)

Edges of the tree:
FG -- F: 5.5
FG -- G: 5.5
DI -- D: 8.5
DI -- I: 8.5
EJ -- E: 10.5
EJ -- J: 10.5
EJL -- L: 19.0
EJL -- EJ: 8.5
AB -- A: 23.0
AB -- B: 23.0
DIH -- H: 29.5
DIH -- DI: 21.0
ABC -- C: 30.2
ABC -- AB: 7.2
ABCEJL -- EJL: 25.3
ABCEJL -- ABC: 14.1
ABCEJLFG -- FG: 55.6
ABCEJLFG -- ABCEJL: 16.8
DIHK -- K: 71.6
DIHK -- DIH: 42.1
ABCEJLFGDIHK -- ABCEJLFG: 17.8
ABCEJLFGDIHK -- DIHK: 7.3

Documentation of the Process:

Step 1:
Merged clusters F and G into FG.
Updated distance matrix:
Cluster	A	B	C	D	E	FG	H	I	J	K	L
A	0	46	48	149	106	134.5	146	151	103	156	96
B	46	0	73	163	80	112.5	167	165	77	126	72
C	48	73	0	141	88	133.5	138	140	88	166	91
D	149	163	141	0	158	178.0	59	17	158	142	153
E	106	80	88	158	0	118.5	173	160	21	115	41
FG	134.5	112.5	133.5	178.0	118.5	0	178.0	180.5	114.5	166.5	115.5
H	146	167	138	59	173	178.0	0	59	168	144	165
I	151	165	140	17	160	180.5	59	0	157	143	156
J	103	77	88	158	21	114.5	168	157	0	107	35
K	156	126	166	142	115	166.5	144	143	107	0	107
L	96	72	91	153	41	115.5	

Finally, we print the constructed evolutionary tree as a list of edges.

In [24]:
# Generate and print the final tree as a list of clusters
final_tree = build_final_tree(edges)
print("\nFinal evolutionary tree as a list of clusters:")
print(final_tree)


Final evolutionary tree as a list of clusters:
[[['F', 'G'], [['L', ['E', 'J']], ['C', ['A', 'B']]]], ['K', ['H', ['D', 'I']]]]


## Conclusion
This assignment demonstrated the effectiveness of the UPGMA algorithm in constructing evolutionary trees from genetic sequence data. By calculating edit distances and iteratively clustering sequences, we revealed evolutionary relationships, with closely related species showing smaller distances and more distant species exhibiting greater divergence. The step-by-step merging process provided clear insights into genetic history, highlighting UPGMA's utility in phylogenetic analysis. This exercise reinforced the importance of computational methods in evolutionary biology and laid the groundwork for exploring more advanced techniques in the future.

